# MNIST Image Recognition Neural Network
#### Chris G Martin
#### CUNY DATA622 - Final Project
#### December 10, 2017

This project was created to design a neural network to identify handwritten numbers from the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

Much of the code and logic used in the first two neural networks come from [Nueral Networks and Deep Learning by Michael Nielsen](http://neuralnetworksanddeeplearning.com/), with modifications and explainations on assumptions and hyper-parameters provided by me. The third network was an experimental one attempting to use Keras.

## Loading the data
Let's start by loading the MNIST data, using the pull steps from the book.

In [1]:
import cPickle
import gzip
import numpy as np

def load_data():
    """
    Used the data from Nielsen's GitHub simply for (1) reproduction and (2) ease of use.
    
    Training set has 50,000 entries,
    Validation set has 10,000 entries,
    And test set has 10,000 entries
    """
    f = gzip.open('C:/Users/itsal/Documents/CUNY/DATA622/Week 15-16/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f)
    f.close()
    return(training_data, validation_data, test_data)

def vectorized_result(j):
    """
    Convert inputs to vector
    Starts by creating empty vector of zeros, one for each digit 0-9       
    For each input j in vector e, label the digits as their binary digit type
    For exampe, a 2 digit returns vector [0, 0, 1, 0, 0, 0, 0, 0, 0]
    """
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def load_data_wrapper():
    """
    Training data = tr_d
    Validation data = va_d
    Test data = te_d
    
    This function reshapes the datasets into a training dataset with a corresponding vector
    of images and a vector with respective binary result indicator. See Neilsen Ch 1
    for reasoning behind format variations of the three sets.
    
    There are 784 pixel inputs (28 x 28).
    """
    tr_d, va_d, te_d = load_data()    
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    #training digit results reshape
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)    
    #validation data reshape
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    #test data reshape
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return(training_data, validation_data, test_data)

In [2]:
training_data, validation_data, test_data = load_data_wrapper()

## Network 1: Stochastic Graident Decent, Cross-Entropy, and Regularization
This neural network utilizes concepts from Chapters 1-3 (Neilsen). It implements stochastic gradient decent for a feedforward neural network, backpropegation, cost estimated by cross-entropy, and regularization. Some experimentation was performed (described in the sections below) to establish hyper-parameters and network weight tuning, so results may vary (for better or, more likely, for worse).

### Feedforward Method, Backpropegation, & Sigmoid
As mentioned, this neural network uses a feedforward method, backpropegation, and sigmoid. Sigmoid is an activation function with inputs (x), weights (w), and bias (b) and can be defined as:

$\sigma(z) = \frac{1}{1 + e^{-z}} = \frac{1}{1 + exp(- \sum_{j} w_{j} x_{j} - b)}$

with

$z = (w * a) + b$

One downside of sigmoid vs other activation functions (i.e. ReLU, Softmax, or tanh) is that activations > |1| lead to a gradient near, if not exactly, zero. This is known as gradient saturation -- when the outputs of the layers in the feedforward pass give an activation too high to backprop a meaningful graident, and restricts the network from learning. The other activation functions also have their own similar or different limitations.

In fact, ReLU (rectified linear units) is shown in Nielsen to be more accurate, and tanh can also improve the network. I will add these functions in the following network, but focus now on sigmoid.

In [3]:
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

The code to use the feedforward method and sigmoid is defined in the network function below as follows:

def feedforward(self, a):<br>
    for b, w in zip(self.biases, self.weights):<br>
        a = sigmoid(np.dot(w, a)+b):<br>
    return a:<br>

The code for backpropegation can be found in the full network code at the end of this section.

### Stochastic Gradient Decent (SGD)

In determining our appropriate weights and biases, we use Stochastic Gradient Decent (SGD) to find a set of weight and biases reducing the cost function (or loss function, or obejective function) of our network as low as possible. SGD uses batches (mini batches) to sample the training set for parameters, vs Gradient Descent's (GD) use of the entire sample. The benefit of using SGD in the MNIST dataset is that there are a fairly decent sample size in the training set (50,000 images), so batching the samples into smaller pieces can speed up the network's learning speed. If there was only a small sample of training data, perhaps it GD would be a more accurate and useful method but using it on the full MNIST datset would add a significant amount of time to run our network.

The code for SGD can be found in the full network code at the end of this section.

### Mini-Batching

As mentioned, SGD uses batches to reduce the time for weight and bias estimations in optimizing our calculation for cost.

### Cross-Entropy Cost
Backpropagation is used to compute partial derivatives $\frac{\partial C}{\partial b}$ and $\frac{\partial C}{\partial w}$ of the cost function (C) in respect to any bias (b) or weight (w). One method of calculating the costs is the quadratic cost, but we will use the cross-entropy cost method for reasons described here. In the case of quadratic cost estimation, the estimation of the partial derivatives are:

$\frac{\partial C}{\partial w} = (a - y)\sigma'(z)x = a \sigma'(z)$

$\frac{\partial C}{\partial b} = (a - y)\sigma'(z) = a \sigma'(z)$

as

$a = \sigma(z)$ where $z = wx + b$

The rate at which a neuron learns is controlled by $\sigma(z) - y$ (= the size of the error of the output); the larger the error, the faster the neuron will learn. Cross-entropy solves this learning slowdown since the $\sigma'(z)$ term is canclled out and the estimation of the partial derivatives are:

$\frac{\partial C}{\partial w} = \frac{1}{n}\sum_{x}x_{j}(\sigma(z) - y)$

$\frac{\partial C}{\partial b} = \frac{1}{n}\sum_{x}(\sigma(z) - y)$

While the quardratic cost function has the form $C = \frac{1}{2 n} \sum_{x} ||y(x) - a^L(x)||^2$, an individual quadradic cost for a single training example (x) is $C = \frac{1}{2}\sum_{j}(y_{j} - a_{j}^L)^2$. n = total number of training examples, x = individual training example, y the corresponding desired output, L is the number of layers in the network, and $a^L$ is the vector of activations output from the network when x is input.

The cost estimate using cross-entry is:

$C = -\frac{1}{n}\sum_{x}[y ln a + (1 - y) ln(1 - a)]$

Cross-Entropy should speed the learning of our network, and theoretically give us better results than the quadratic method.

In [4]:
class CrossEntropyCost(object):
    """
    Class for cost-entrophy cost: class is used to return two different measures
    (1) CrossEntropyCost.fn computes the cost, how well output activation (a) matches expected output (y)
    (2) CrossEntropyCost.delta computes the delta of the output error
    """
    @staticmethod
    def fn(a, y):
        return np.sum(np.nan_to_num(-y * np.log(a) - (1-y) * np.log(1-a)))
    
    @staticmethod
    def delta(z, a, y):
        return (a-y)

### Regularization
Regularization is a method to reduce overfitting. The regularization technique used in this model is L2 regularization, in which an extra term (regularization term) is added to the cost function. This technique is also known as weight decay for reasons shown here. For our cross-entropy cost we can see the change as:

$C = -\frac{1}{n} \sum_{xj} [y_{j} ln a_{j}^L + (1 - y_{j})(ln(1 - a_{j}^L)] + \frac{\lambda}{2n} \sum_{w} w^2$

With our regularization term:

$\frac{\lambda}{2n} \sum_{w} w^2$

More simply, regularization changes cost in general as:

$C = C_{0} + \frac{\lambda}{2n} \sum_{w} w^2$

As it can be seen, $\lambda$ is an important term since the value of can determine the impact regularization has to cost. With a large $\lambda$, small weights are prefered since the impact on cost is reduced. With a small $\lambda$, larger weights are preferred since it will impact the cost. The partial dervivatives of $\frac{\partial C}{\partial b}$ and $\frac{\partial C}{\partial w}$ now become (for use in backpropegation):

$\frac{\partial C}{\partial w} = \frac{\partial C_{0}}{\partial w} + \frac{\lambda}{n}w$

$\frac{\partial C}{\partial b} = \frac{\partial C_{0}}{\partial b}$

There is no change to bias, but with gradient descent (and, more importantly, SGD) our weight begins to rescale (decay) at a factor of $1 - \frac{\eta \lambda}{n}.

Other regularization types include L1 regularization (which changes the addition to cost of $\frac{\lambda}{n}\sum{w}$ the absolute value of weights $\frac{\lambda}{n}\sum{w}$, effectively changing the impact of extremely large/small weights.

Dropout is a method of regularization by which neurons in the network are randomly (and temporarily) dropped from the network during forward-propegation and backpropegation, and the pattern is continously repeated. Weights and biases are thusly changed under different conditions and (presumably) averaged.

Artifically expanding the training set is also an interesting method, as the training set can be increased by slightly modifying the inputs. For our MNIST example, we can increase the training set by slightly 'tilting' our images (i.e. 15 degrees), or osciliating generated digits (to mimic hand spasms), or skewing images.

The code for regularization can be found in the full network code at the end of this section.

### Weight Initialization
The topic of weight initialization was quickly covered by Nielsen, so outside sources such as a blog post by [Andre Perunicic on Neural Network Weight Initialization](https://intoli.com/blog/neural-network-initialization/), a [StackExchange question on Initial Weights](https://intoli.com/blog/neural-network-initialization/), a blog post by [Isaac Changhau called Weight Initialization in Artificial Neural Networks](https://isaacchanghau.github.io/2017/05/24/Weight-Initialization-in-Artificial-Neural-Networks/), a write-up by [Siddharth Krishna Kumar at Cornell University](https://arxiv.org/abs/1704.08863), and a post on [Xavier initialization by Prateek Joshi](https://prateekvjoshi.com/2016/03/29/understanding-xavier-initialization-in-deep-neural-networks/)  served as side-research into the subject.

One apparently popular method of initlization is Xavier initilization. A Guassian distribution has a mean of zero and a specific, finance variance. An Xavier initiliazation itiliazes the weights across the layers and keeps the variance through each layer the same, through a Gaussian distribution. The forumla for the variance using backpropegation would be:

$Var(W_{i}) = \frac{2}{n_{in} + n_{out}}$

with a more simplified (and possibly refined?) forumla being:

$Var(W) = \frac{2}{n}$

The code for weight initialization can be found in the full network code at the end of this section.

### Hyper Parameter Tuning
Hyper parameters are extremely important to the model, and tuning can be performed in a number of ways. I can even imagine a quite simple Monte Carlo function to run several experiements on tuning the parameters until an 'optimal' parameter is found (or, the best of the rest parameters which seems more likely). These parameters could be the difference 

The primary parameters I adjusted were $\eta$ and $\lambda$, by running the network several times and reducing the number of samples in our evaluation data and training data sets to 100 and 1000, respectively. Reducing the number of samples allowed me to quickly tune the parameters, since the network was significantly sped up due to reduced input sizes. Tuning was performed by first setting the parameters to extremes (such as $\eta$=1000 and $\lambda$=1000, then reducing or increasing the parameter until it seems like a good fit, then switching to the other parameter and increasing or reducing the parameter until a good fit seemed in range, then rinse and repeat.

A major obstacle I found was that I could find parameters that fit the training data very well (~99% accuracy), but the evaulation set was far lower (~80% accuracy) which indicated overfitting. I also found that spme well performing parameters on the shrunked set did not perform well with the full set.

Final hyper parameters are set in the code to run the newtork as shown at the end of this section.

### Accuracy of the Network and Total Cost
The next question we need to ask is, how do we track the performance of our network? Accuracy can be tracked by counting the number of correctly identified images, while total cost is monitored as a means for efficiency.

The code for accuracy and total cost can be found in the full network code at the end of this section.

### Neural Network
Using the data loaded early, the CrossEntropyCost class defined earlier, the sigmoid and sigmoid prime functions, and the discussions above, here is the neural network, with minor modifications to the [Nielsen code (network2.py)](https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/src/network2.py):

In [5]:
import json
import random

class Network1(object):
    
    def __init__(self, sizes, cost=CrossEntropyCost):
        """
        Sizes contains the number of neurons in the respective layers of the network.
        For example [2, 3, 1] is a three-layer network with 2 neurons in the first layer,
        3 in the second layer, and 1 neuron in the third layer.
        
        xavier_initilizer initilizes the biases and weights of the network randomly.
        default_weight_initializer uses Gaussian distribution.
        large_weight_initilizer standardize or does not reduce the weights.
        These are all included for comparison
        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.xavier_initilizer()
        self.cost = cost
        
    def xavier_initilizer(self):
        """
        Initialize weights using a Gaussian distribution with mean 0.
        The first layer is assumed to be an input layer, so no bias is set.
        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [(np.random.randn(y, x)*2)/(np.sqrt(x))
                       for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def default_weight_initializer(self):
        """
        Initialize weights using a Gaussian distribution with mean 0 std of 1.
        The first layer is assumed to be an input layer, so no bias is set.
        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                       for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initilizer(self):
        """
        Initialize weights using a Gaussian distribution with mean 0.
        The first layer is assumed to be an input layer, so no bias is set.
        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                       for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a):
        """
        Feed forward function, using sigmoid.
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, lmbda = 0.0,
            evaluation_data=None, monitor_evaluation_cost=False, monitor_evaluation_accuracy=False,
            monitor_training_cost=False, monitor_training_accuracy=False):
        """
        Stochastic Gradient Descent using batches as defined by mini_batch_size.
        Evaluation data is optional, and can be included when the network is run.
        lmbda is the regularization parameter.
        Cost and accuracy can be tracked by marking 'True' for evalution and training data
        with costs per epoch, with the first tuple being the list at the end of the epoch.
        """
        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)
        #create empty vectors to be filled in if monitors are true
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, lmbda, len(training_data))
            print "Epoch %s training complete" % j
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print "Cost on training data: {}".format(cost)
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print "Accuracy on training data: {} / {}".format(accuracy, n)
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print "Cost on evaluation data: {}".format(cost)
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print "Accuracy on evaluation data: {} / {}".format(self.accuracy(evaluation_data), n_data)
            print
        return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy
    
    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        """
        Updates the network's weights and biases by applying GD using backpropegation to a single mini batch.
        The mini_batch is a list of tuples (x,y).
        eta is the learning rate.
        n is the total size of the training data set.
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x,y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.baises = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        """
        Returns a tuple representing the gradient for the cost function C_x.
        nabla_b and nabla_w is a list of arrays for each layer.
        """
        #b=bias, w=weight
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        #feedforward
        activation = x
        #store all activations by layer
        activations = [x]
        #store all z vectors by layer
        zs = []
        for b, w in zip(self.biases, self.weights):
            #z = w * a + b
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        #backward pass
        #-1 indicates the last value
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in xrange(2, self.num_layers):
            #-l as in L, not 1
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def accuracy(self, data, convert=False):
        """
        Returns the correct number of inputs identified by the network.
        Convert should be set to False if the data set is validation or test data, and True if training data.
        As seen in the data load, y is different in the training data set, so conversion is made here.
        """
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                      for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)
                      for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)
    
    def total_cost(self, data, lmbda, convert=False):
        """
        Returns the total cost for the data set in data. See 'accuracy' for convert.
        """
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
        cost += 0.5 * (lmbda/len(data)) * sum(np.linalg.norm(w)**2 for w in self.weights)
        return cost
    
    def save(self, filename):
        """
        Saves the neural network to the file 'filename'.
        """
        data = {"sizes": self.sizes,
               "weights": [w.tolist() for w in self.weights],
               "biases": [b.tolist() for b in self.biases],
               "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.close()

### Running the Network
With our network built, we can run it and see how it performs. Remember, our data was already loaded at the beginning.

In [2]:
net = Network1([784, 10])#, cost=CrossEntropyCost)
evaluation_cost, evaluation_accuracy, training_cost, training_accuracy = net.SGD(
    training_data, 40, 10, 0.156, lmbda=2.2, evaluation_data=validation_data,
            monitor_evaluation_accuracy=True, monitor_evaluation_cost=True,
            monitor_training_accuracy=True, monitor_training_cost=True)

NameError: name 'Network1' is not defined

As you can see from the results, it looks like I was able to get an accuracy on the training data of ~92% correct identification, and evaluation accuracy of ~92%. This is pretty good, but not even close to the results Nielsen obtained (~98%). The weakness could be in our hyper parameters, xavier initialization, or epoch sizes.

## Network 2: Theanos Convolutional Neural Network
The Convolutional Neural Network expands the neural network by piecing together pixels (for image data like what we're using here) into a feature map representing block images. This particular neural network uses pooling methods to create hidden layers of pooled pixel inputs. Most of the code here is from Nielsen's book.

### Activation Functions
As mentioned in the previous network description, ReLU and tanh activation may outperform the sigmoid activation function we used. Here we will define both functions, and add linear activation.

In [3]:
import theano

def linear(z): return z
def ReLU(z): return T.maximum(0.0, z)

#pre-defined sigmoid activation
from theano.tensor.nnet import sigmoid
#pre-defined tanh activation
from theano.tensor import tanh

C:\Program Files\Anaconda2\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."


### GPU, Parallel, and Cloud Processing
One advantage of using Theano is that it enables GPU to speed up processing along with CPU. Other ways to improve processing speeds are to use parallel processing techniques or cloud processing. For the sake of simplicity and time (the irony, I know) and due to the limitations of my laptop (which is on its last legs), I will simply use CPU processing but add the option for GPU here:

In [4]:
GPU = False
if GPU:
    print "Trying to run under GPU. If this is not desired then set GPU to False."
    try: theano.config.device = 'gpu'
    except: pass
    theano.config.floatX = 'float32'
else:
    print "Running with CPU. If this is not desired then set GPU to True."

Running with a CPU. If this is not desired then set GPU to True.


### Shared Variables
The data set has already been loaded in the beginning, but Theano uses shared variables allowing the data to be copied to the GPU.

In [5]:
import theano.tensor as T

def load_data_shared(filename="C:/Users/itsal/Documents/CUNY/DATA622/Week 15-16/mnist.pkl.gz"):
    f = gzip.open(filename, 'rb')
    training_data, validation_data, test_data = cPickle.load(f)
    f.close()
    def shared(data):
        """
        Shared variables
        """
        shared_x=theano.shared(np.asarray(data[0], dtype=theano.config.floatX), borrow=True)
        shared_y=theano.shared(np.asarray(data[1], dtype=theano.config.floatX), borrow=True)
        return shared_x, T.cast(shared_y, "int32")
    return [shared(training_data), shared(validation_data), shared(test_data)]

### Convolutional Neural Network

In [9]:
from theano.tensor.nnet import conv2d
from theano.tensor.nnet import softmax
from theano.tensor import shared_randomstreams
from theano.tensor.signal import pool


class Network2(object):
    
    def __init__(self, layers, mini_batch_size):
        """
        
        """
        self.layers = layers
        self.mini_batch_size = mini_batch_size
        self.params = [param for layer in self.layers for param in layer.params]
        self.x = T.matrix("x")
        self.y = T.ivector("y")
        init_layer = self.layers[0]
        init_layer.set_inpt(self.x, self.x, self.mini_batch_size)
        for j in xrange(1, len(self.layers)):
            prev_layer, layer = self.layers[j-1], self.layers[j]
            layer.set_inpt(prev_layer.output, prev_layer.output_dropout, self.mini_batch_size)
        self.output = self.layers[-1].output
        self.output_dropout = self.layers[-1].output_dropout
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, validation_data, test_data, lmbda=0.0):
        """
        Stochastic Gradient Descent with mini-batches
        """
        training_x, training_y = training_data
        validation_x, validation_y = validation_data
        test_x, test_y = test_data
        #number of mini-batches for each type of data
        num_training_batches = size(training_data)/mini_batch_size
        num_validation_batches = size(validation_data)/mini_batch_size
        num_test_batches = size(test_data)/mini_batch_size
        #regularized cost, symbolic gradients, and updates
        l2_norm_squared = sum([(layer.w**2).sum() for layer in self.layers])
        cost = self.layers[-1].cost(self) + 0.5 * lmbda * l2_norm_squared / num_training_batches
        grads = T.grad(cost, self.params)
        updates = [(param, param-eta*grad) for param, grad in zip(self.params, grads)]
        #compute accruacy in validation and test mini-batches
        i = T.lscalar()
        
        train_mb = theano.function(
            [i], cost, updates=updates,
            givens={
                self.x:
                training_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size],
                self.y:
                training_y[i*self.mini_batch_size: (i+1)*self.mini_batch_size]
            })

        #train_mb = theano.function([i], cost, updates=updates, givens={
        #        self.x: training_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size],
        #        self.y: training_y[i*self.mini_batch_size: (i+1)*self.mini_batch_size]})
        validate_mb_accuracy = theano.function([i], self.layers[-1].accuracy(self.y), givens={
                self.x: validation_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size],
                self.y: validation_y[i*self.mini_batch_size: (i+1)*self.mini_batch_size]})
        test_mb_accuracy = theano.function([i], self.layers[-1].accuracy(self.y), givens={
                self.x: test_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size],
                self.y: test_y[i*self.mini_batch_size: (i+1)*self.mini_batch_size]})
        self.test_mb_predictions = theano.function([i], self.layers[-1].y_out, givens={
                self.x: test_x[i*self.mini_batch_size: (i+1)*self.mini_batch_size]})
        #training
        best_validation_accuracy = 0.0
        for epoch in xrange(epochs):
            for minibatch_index in xrange(num_training_batches):
                iteration = num_training_batches*epoch + minibatch_index
                if iteration % 1000 == 0:
                    print("Training mini-batch number [0]".format(iteration))
                cost_ij = train_mb(minibatch_index)
                if (iteration+1) % num_training_batches == 0:
                    validation_accuracy = np.mean([validate_mb_accuracy(j) for j in xrange(num_validation_batches)])
                    print("Epoch [0]: validation accuracy {1:.2%}".format(epoch, validation_accuracy))
                    if validation_accuracy >= best_validation_accuracy:
                        print("This is the best validation accuracy to date.")
                        best_validation_accuracy = validation_accuracy
                        best_iteration = iteration
                        if test_data:
                            test_accuracy = np.mean([test_mb_accuracy(j) for j in xrange(num_test_batches)])
                            print("The corresponding test accuracy is {0:.2%}".format(test_accuracy))
            print("Finished training network.")
            print("Best validation accuracy of {0:.2%} obtained at iteration {1}".format(best_validation_accuracy, best_iteration))
            print("Corresponding test accuracy of {0:.2%}".format(test_accuracy))

class ConvPoolLayer(object):
    """
    Creates a convolutional layer and max-pooling layer.
    The two layers are combined into a single class for consistency with Nielsen.
    """
    
    def __init__(self, filter_shape, input_shape, poolsize=(2,2), activation_fn=sigmoid):
        """
        Default using a pool of 2x2 and sigmoid activation
        """
        self.filter_shape = filter_shape
        self.input_shape = input_shape
        self.poolsize = poolsize
        self.activation_fn = activation_fn
        #initialization
        n_out = (filter_shape[0]*np.prod(filter_shape[2:])/np.prod(poolsize))
        self.w = theano.shared(np.asarray(np.random.normal(loc=0, scale=np.sqrt(1.0/n_out), size=filter_shape),
                                         dtype=theano.config.floatX),
                              borrow=True)
        self.b = theano.shared(np.asarray(np.random.normal(loc=0, scale=1.0, size=(filter_shape[0],)),
                                         dtype=theano.config.floatX),
                              borrow=True)
        self.params = [self.w, self.b]
        
    def set_inpt(self, inpt, inpt_dropout, mini_batch_size):
        self.inpt = inpt.reshape(self.input_shape)
        conv_out = conv2d(input=self.inpt, filters=self.w, filter_shape=self.filter_shape,
                              input_shape=self.input_shape)
        pooled_out = pool.pool_2d(input=conv_out, ws=self.poolsize, ignore_border=True)
        self.output = self.activation_fn(pooled_out + self.b.dimshuffle('x', 0, 'x', 'x'))
        self.output_dropout = self.output

class FullyConnectedLayer(object):
    """
    Fully Connected Layer
    """
    
    def __init__(self, n_in, n_out, activation_fn=sigmoid, p_dropout=0.0):
        self.n_in = n_in
        self.n_out = n_out
        self.activation_fn = activation_fn
        self.p_dropout = p_dropout
        #initalizes weights and biases
        self.w = theano.shared(
            np.asarray(
                np.random.normal(
                    loc=0.0, scale=np.sqrt(1.0/n_out), size=(n_in, n_out)),
                dtype=theano.config.floatX),
            name='w', borrow=True)
        self.b = theano.shared(
            np.asarray(np.random.normal(loc=0.0, scale=1.0, size=(n_out,)),
                       dtype=theano.config.floatX),
            name='b', borrow=True)

        #self.w = theano.shared(np.asarray(np.random.normal(loc=0.0, scale=np.sqrt(1/n_out), size=(n_in, n_out)),
        #                                  dtype=theano.config.floatX),
        #                       name='w', borrow=True)
        #self.b = theano.shared(np.asarray(np.random.normal(loc=0.0, scale=1.0, size=(n_out,)),
        #                                 dtype=theano.config.floatX),
        #                       name='b', borrow=True)
        self.params = [self.w, self.b]
    
    def set_inpt(self, inpt, inpt_dropout, mini_batch_size):
        """
        input
        """
        self.inpt = inpt.reshape((mini_batch_size, self.n_in))
        self.output = self.activation_fn((1-self.p_dropout)*T.dot(self.inpt, self.w) + self.b)
        self.y_out = T.argmax(self.output, axis=1)
        self.inpt_dropout = dropout_layer(inpt_dropout.reshape((mini_batch_size, self.n_in)), self.p_dropout)
        self.output_dropout = self.activation_fn(T.dot(self.inpt_dropout, self.w) + self.b)
    
    def accuracy(self, y):
        """
        Accuracy of mini-batch
        """
        return T.mean(T.eq(y, self.y_out))

class SoftmaxLayer(object):
    """
    Softmax Layer
    """
    
    def __init__(self, n_in, n_out, p_dropout=0.0):
        self.n_in = n_in
        self.n_out = n_out
        self.p_dropout = p_dropout
        #initializes wights and biases
        self.w = theano.shared(np.zeros((n_in, n_out), dtype=theano.config.floatX), name='w', borrow=True)
        self.b = theano.shared(np.zeros((n_out,), dtype=theano.config.floatX), name='b', borrow=True)
        self.params = [self.w, self.b]
    
    def set_inpt(self, inpt, inpt_dropout, mini_batch_size):
        """
        input
        """
        self.inpt = inpt.reshape((mini_batch_size, self.n_in))
        self.output = softmax((1-self.p_dropout)*T.dot(self.inpt, self.w) + self.b)
        self.y_out = T.argmax(self.output, axis=1)
        self.inpt_dropout = dropout_layer(inpt_dropout.reshape((mini_batch_size, self.n_in)), self.p_dropout)
        self.output_dropout = softmax(T.dot(self.inpt_dropout, self.w) + self.b)
    
    def cost(self, net):
        """
        Returns the cost (log-likelihood).
        """
        return -T.mean(T.log(self.output_dropout)[T.arange(net.y.shape[0]), net.y])
    
    def accuracy(self, y):
        """
        Returns the accuracy of the mini-batch.
        """
        return T.mean(T.eq(y, self.y_out))

def size(data):
    """
    Size of the data set.
    """
    return data[0].get_value(borrow=True).shape[0]

def dropout_layer(layer, p_dropout):
    """
    Dropout Layer
    """
    srng = shared_randomstreams.RandomStreams(np.random.RandomState(0).randint(999999))
    mask = srng.binomial(n=1, p=1-p_dropout, size=layer.shape)
    return layer*T.cast(mask, theano.config.floatX)

### Running the Network
The code above allows us to specify the layers we would like to add, the type of activation in the layers, and the mini-batch size, as well as several other parameters.

In [7]:
training_data, validation_data, test_data = load_data_shared()
mini_batch_size = 10

In [ ]:
net = Network2([
        ConvPoolLayer(input_shape=(mini_batch_size, 1, 28, 28),
                     filter_shape=(20, 1, 5, 5),
                     poolsize=(2,2),
                     activation_fn=ReLU),
        ConvPoolLayer(input_shape=(mini_batch_size, 20, 12, 12),
                     filter_shape=(40, 20, 5, 5),
                     poolsize=(2,2),
                     activation_fn=ReLU),
        FullyConnectedLayer(n_in=40*4*4, n_out=1000, activation_fn=ReLU, p_dropout=0.5),
        SoftmaxLayer(n_in=1000, n_out=10, p_dropout=0.5)],
              mini_batch_size)

net.SGD(training_data, 60, mini_batch_size, 0.03, validation_data, test_data, lmbda=0.1)

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:114: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
C:\Program Files\Anaconda2\lib\site-packages\theano\tensor\nnet\conv.py:98: UserWarning: theano.tensor.nnet.conv.conv2d is deprecated. Use theano.tensor.nnet.conv2d instead.
  warnings.warn("theano.tensor.nnet.conv.conv2d is deprecated."


Training mini-batch number [0]


This network ran extremely slowly, and I blame the quality of my computer and personal impatience. Debugging fixed several errors, and fixing depreciating/depreciated theanos modules fixed several more. As such, I did not change the hyper parameters as used in Nielsen. My best performance marked 97.7% accuracy. A far better result than my previous attempt.

## Network 3: Keras



## Loading a Network
The functioned defined below can be used to load a saved neural network.

In [ ]:
import sys

def load(filename):
    """
    Load a neural network from the file 'filename'.
    """
    f = open(fileame, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net